# Setup project

In [1]:
import pandas as pd
import numpy as np

In [133]:
df_2010 = pd.read_csv('reces_2010_canton_ville_D.csv', sep=';', low_memory=False)

In [3]:
df_cantons = pd.read_csv('cantons_cleaned.csv')

In [198]:
DEPARTEMENTS = ['16',
                '17',
                '19',
                '23',
                '24',
                '33',
                '40',
                '47',
                '64',
                '79',
                '86',
                '87']
DEPARTEMENTS_INT = [int(numeric_string) for numeric_string in DEPARTEMENTS]

# Study dataset

In [11]:
df_2010.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4311337 entries, 0 to 4311336
Data columns (total 89 columns):
 #   Column       Dtype  
---  ------       -----  
 0   CANTVILLE    int64  
 1   NUMMI        object 
 2   AEMMR        object 
 3   AGED         int64  
 4   AGER20       int64  
 5   AGEREV       int64  
 6   AGEREVQ      int64  
 7   ANAI         int64  
 8   ANEMR        object 
 9   APAF         object 
 10  ARM          object 
 11  ASCEN        object 
 12  BAIN         object 
 13  BATI         object 
 14  CATIRIS      object 
 15  CATL         object 
 16  CATPC        int64  
 17  CHAU         object 
 18  CHFL         object 
 19  CHOS         object 
 20  CLIM         object 
 21  CMBL         object 
 22  COUPLE       int64  
 23  CS1          int64  
 24  CUIS         object 
 25  DEPT         int64  
 26  DEROU        object 
 27  DIPL         object 
 28  DNAI         object 
 29  EAU          object 
 30  EGOUL        object 
 31  ELEC         object 
 32

In [6]:
df_2010.head()

,CANTVILLE,NUMMI,AEMMR,AGED,AGER20,AGEREV,AGEREVQ,ANAI,ANEMR,APAF,...,TP,TRANS,TRIRIS,TYPC,TYPFC,TYPL,TYPMC,TYPMR,VOIT,WC
0,101,1,6,58,64,57,55,1950,05,0,...,Z,Z,ZZZZZZ,2,Z,1,1,12,1,Z
1,101,2,6,55,54,54,50,1955,05,2,...,1,4,ZZZZZZ,1,2,1,4,41,3,Z
2,101,2,6,55,54,54,50,1955,05,1,...,1,4,ZZZZZZ,1,2,1,4,41,3,Z
3,101,3,9,45,54,44,40,1963,01,0,...,1,2,ZZZZZZ,2,Z,1,1,12,1,Z
4,101,4,9,58,64,57,55,1953,01,0,...,1,5,ZZZZZZ,3,Z,2,1,11,1,Z


In [7]:
df_2010['CANTVILLE'].value_counts()

CANTVILLE
6999    183960
3199    168310
3399     91213
4299     64985
3898     61222
         ...  
8103       158
913        137
3893       116
4398        42
8140        35
Name: count, Length: 1082, dtype: int64

In [8]:
df_cantons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Département      255 non-null    int64
 1   Canton-ou-ville  255 non-null    int64
dtypes: int64(2)
memory usage: 4.1 KB


# Start working on dataset

In [128]:
# add empty column for storing future department INSEE code
#df_2010["departement"] = np.nan

In [134]:
test = df_2010.sample(n=10)

In [154]:
test['annee'] = 2010

In [214]:
test

,CANTVILLE,NUMMI,AEMMR,AGED,AGER20,AGEREV,AGEREVQ,ANAI,ANEMR,APAF,...,TRANS,TRIRIS,TYPC,TYPFC,TYPL,TYPMC,TYPMR,VOIT,WC,annee
1551210,3356,5543,7,45,54,45,45,1964,04,0,...,4,330811,3,Z,2,1,12,1,Z,2010
2125233,4004,1314,9,25,24,24,20,1987,00,0,...,4,ZZZZZZ,3,Z,2,1,11,1,Z,2010
3995030,7499,1309,9,7,10,6,5,2002,02,3,...,Z,740021,3,2,2,4,41,1,Z,2010
4219987,8717,407,9,5,5,4,0,2006,02,3,...,Z,ZZZZZZ,1,2,1,4,41,2,Z,2010
4246735,8735,549,9,56,64,55,55,1956,03,2,...,4,ZZZZZZ,1,2,1,4,41,2,Z,2010
2464461,4312,394,7,61,64,60,60,1947,04,1,...,Z,ZZZZZZ,1,2,1,4,44,1,Z,2010
2106714,3899,1076,9,79,79,78,75,1929,02,0,...,Z,380411,3,Z,2,1,12,0,Z,2010
2213840,4098,2933,9,5,5,4,0,2005,02,3,...,Z,400061,1,1,1,3,32,1,Z,2010
598250,2327,33,7,58,64,57,55,1952,05,2,...,Z,ZZZZZZ,1,2,1,4,44,2,Z,2010
2272380,4213,738,8,16,17,15,15,1992,03,3,...,Z,ZZZZZZ,1,2,1,4,41,2,Z,2010


In [245]:
def replace_letter_to_int(row, column):
    ascii = ord(row[column])
    row[column].replace(regex=r'[A-Z]', value=ascii)
    return row

In [159]:
truc = test[test['CANTVILLE'].isin(df_cantons['Canton-ou-ville'])]
truc

,CANTVILLE,NUMMI,AEMMR,AGED,AGER20,AGEREV,AGEREVQ,ANAI,ANEMR,APAF,...,TRANS,TRIRIS,TYPC,TYPFC,TYPL,TYPMC,TYPMR,VOIT,WC,annee
2125233,4004,1314,9,25,24,24,20,1987,00,0,...,4,ZZZZZZ,3,Z,2,1,11,1,Z,2010
4219987,8717,407,9,5,5,4,0,2006,02,3,...,Z,ZZZZZZ,1,2,1,4,41,2,Z,2010
2213840,4098,2933,9,5,5,4,0,2005,02,3,...,Z,400061,1,1,1,3,32,1,Z,2010


In [180]:
merged_df = pd.merge(truc, df_cantons, how='left', left_on='CANTVILLE', right_on='Canton-ou-ville')

In [181]:
merged_df = merged_df.rename(columns={
    'Département': 'departement'
})

In [195]:
merged_df = merged_df.drop(columns=[
    'CANTVILLE', 
    'Canton-ou-ville',
    'NUMMI',
    'AEMMR',
    'AGEREV',
    'AGER20',
    'AGEREVQ',
    'ANAI',
    'ANEMR',
    'APAF',
    'ARM',
], errors='ignore')

In [251]:
merged_df = merged_df.apply(replace_letter_to_int, axis=1, result_type='broadcast', column='CATIRIS')

TypeError: replace_letter_to_int() got an unexpected keyword argument 'axis'

In [216]:
merged_df['BATI'].va£lue_counts()

BATI
Z    3
Name: count, dtype: int64

In [217]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 79 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   AGED         3 non-null      int64 
 1   ASCEN        3 non-null      object
 2   BATI         3 non-null      object
 3   CATIRIS      3 non-null      object
 4   CATL         3 non-null      object
 5   CATPC        3 non-null      int64 
 6   CHAU         3 non-null      object
 7   CHFL         3 non-null      object
 8   CHOS         3 non-null      object
 9   CLIM         3 non-null      object
 10  CMBL         3 non-null      object
 11  COUPLE       3 non-null      int64 
 12  CS1          3 non-null      int64 
 13  CUIS         3 non-null      object
 14  DEPT         3 non-null      int64 
 15  DEROU        3 non-null      object
 16  DIPL         3 non-null      object
 17  DNAI         3 non-null      object
 18  EAU          3 non-null      object
 19  EGOUL        3 non-null      obje

In [233]:
merged_df.head()

,AGED,ASCEN,BATI,CATIRIS,CATL,CATPC,CHAU,CHFL,CHOS,CLIM,...,TRIRIS,TYPC,TYPFC,TYPL,TYPMC,TYPMR,VOIT,WC,annee,departement
0,25,2,Z,H,1,0,Z,3,Z,Z,...,ZZZZZZ,3,Z,2,1,11,1,Z,2010,40
1,5,2,Z,Z,1,0,Z,2,Z,Z,...,ZZZZZZ,1,2,1,4,41,2,Z,2010,87
2,5,2,Z,H,1,0,Z,2,Z,Z,...,400061,1,1,1,3,32,1,Z,2010,40
